In [ ]:
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git

In [1]:
!pip install open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.6 MB/s eta 0:00:00


In [2]:
import open_clip
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commo

In [3]:
import json
import requests
import torch
from torch import nn
import numpy as np
from zipfile import ZipFile
from sklearn.decomposition import PCA
from torchvision.transforms import InterpolationMode
from torchvision.transforms.functional import resize, normalize
import open_clip
from open_clip import create_model_and_transforms, tokenize
import pandas as pd 

from tqdm.notebook import tqdm, trange

# import clip
from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df_caption = pd.read_csv('/kaggle/input/multi-model-dataset/For validation/superAI/caption_id.csv')
df_caption.head()

,caption_id,caption
0,49,"The man, with a straw hat and a white mask, is..."
1,85,"Wearing a straw hat and a white face mask, the..."
2,86,"With a straw hat and a white mask on, the man ..."
3,61,"The man, donning a straw hat and a white face ..."
4,95,"Sporting a straw hat and a white mask, the mal..."


In [5]:
df_image = pd.read_csv('/kaggle/input/multi-model-dataset/For validation/superAI/img_id.csv')
df_image['file_path']=df_image['file_path'].apply(lambda x : '/kaggle/input/multi-model-dataset/For validation/superAI/imgs/'+x)
df_image['predict'] = [i for i in range(len(df_image))]
df_image.head()

,file_path,identity_id,predict
0,/kaggle/input/multi-model-dataset/For validati...,38,0
1,/kaggle/input/multi-model-dataset/For validati...,38,1
2,/kaggle/input/multi-model-dataset/For validati...,38,2
3,/kaggle/input/multi-model-dataset/For validati...,38,3
4,/kaggle/input/multi-model-dataset/For validati...,39,4


In [6]:
df_sub = pd.read_csv('/kaggle/input/multi-model-dataset/For validation/superAI/sample_submission.csv')
df_sub.head()

,caption_id,identity_id
0,35,41.0
1,69,38.0
2,42,44.0
3,17,NaN
4,76,NaN


In [7]:
df_all = df_sub.merge(df_caption,how='left',on=['caption_id'])
df_all['predict'] = [i for i in range(len(df_all))]
df_all

,caption_id,identity_id,caption,predict
0,35,41.0,"Sporting a white mask, the male individual is ...",0
1,69,38.0,"The man, wearing a straw hat and a white mask,...",1
2,42,44.0,A powerful man is dressed in a red short-sleev...,2
3,17,NaN,A man is dressed in a red and white short-slee...,3
4,76,NaN,With a yellow safety helmet and a white mask o...,4
...,...,...,...,...
95,29,NaN,A towering man is dressed in a red short-sleev...,95
96,8,NaN,A strong man is seen wearing a red short-sleev...,96
97,9,NaN,"Donning a white face mask, the male individual...",97
98,64,NaN,A boy in black shorts and short sleeves with a...,98


In [8]:
# Load model and transforms
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = create_model_and_transforms('xlm-roberta-large-ViT-H-14', 'frozen_laion5b_s13b_b90k', device=device)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
model.eval()
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 1,193,014,785
Context length: 514
Vocab size: 250002


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k')
tokenizer = open_clip.get_tokenizer('xlm-roberta-large-ViT-H-14')

image = [preprocess(Image.open(path)) for path in tqdm(df_image['file_path'])]
image_input = torch.tensor(np.stack(image)).cuda()
list_text=df_all['caption'].tolist()
# image = preprocess(Image.open(df_image['file_path'][0])).unsqueeze(0).to(device)


  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
image_input = torch.tensor(np.stack(image)).to(device)

In [12]:
print(list_text[0])
text = tokenizer(list_text[0]).to(device)
text

Sporting a white mask, the male individual is dressed in a yellow outerwear, black undershirt, black trousers, and black athletic shoes. He is holding a yellow plush toy in his hands.


tensor([[     0,   4706,    214,     10,  35011,   7021,      4,     70,  11280,
          11651,     83,  40989,    297,     23,     10, 205811,   1810,     56,
          87690,      4,  22556,   1379,  38184,      4,  22556,  36113,      7,
           1314,      4,    136,  22556,     99,   7057,   9523, 148100,      5,
           1529,     83, 104064,     10, 205811,   1001,    127,  72713,     23,
           1919,  44540,      5,      2,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1]], device='cuda:0')

In [13]:
text = tokenizer(list_text[0]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

print("Label probs:", torch.from_numpy(similarity).argmax(dim=1).item())  


Label probs: 15


In [14]:
for i,column in tqdm(df_all.iterrows(),total=100) :
    text = tokenizer(column['caption']).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
        
    df_all.loc[i,'predict']=torch.from_numpy(similarity).argmax(dim=1).item()

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
df_all

,caption_id,identity_id,caption,predict
0,35,41.0,"Sporting a white mask, the male individual is ...",15
1,69,38.0,"The man, wearing a straw hat and a white mask,...",3
2,42,44.0,A powerful man is dressed in a red short-sleev...,26
3,17,NaN,A man is dressed in a red and white short-slee...,32
4,76,NaN,With a yellow safety helmet and a white mask o...,6
...,...,...,...,...
95,29,NaN,A towering man is dressed in a red short-sleev...,23
96,8,NaN,A strong man is seen wearing a red short-sleev...,26
97,9,NaN,"Donning a white face mask, the male individual...",15
98,64,NaN,A boy in black shorts and short sleeves with a...,40


In [16]:
df_sub = df_all.merge(df_image,how='left',on=['predict'])
df_sub

,caption_id,identity_id_x,caption,predict,file_path,identity_id_y
0,35,41.0,"Sporting a white mask, the male individual is ...",15,/kaggle/input/multi-model-dataset/For validati...,41
1,69,38.0,"The man, wearing a straw hat and a white mask,...",3,/kaggle/input/multi-model-dataset/For validati...,38
2,42,44.0,A powerful man is dressed in a red short-sleev...,26,/kaggle/input/multi-model-dataset/For validati...,44
3,17,NaN,A man is dressed in a red and white short-slee...,32,/kaggle/input/multi-model-dataset/For validati...,46
4,76,NaN,With a yellow safety helmet and a white mask o...,6,/kaggle/input/multi-model-dataset/For validati...,39
...,...,...,...,...,...,...
95,29,NaN,A towering man is dressed in a red short-sleev...,23,/kaggle/input/multi-model-dataset/For validati...,43
96,8,NaN,A strong man is seen wearing a red short-sleev...,26,/kaggle/input/multi-model-dataset/For validati...,44
97,9,NaN,"Donning a white face mask, the male individual...",15,/kaggle/input/multi-model-dataset/For validati...,41
98,64,NaN,A boy in black shorts and short sleeves with a...,40,/kaggle/input/multi-model-dataset/For validati...,48


In [21]:
df_sub['identity_id_y'].value_counts()

46    11
41     8
38     8
44     8
39     8
40     8
45     8
47     8
43     8
42     7
48     6
49     5
51     3
55     2
62     1
50     1
Name: identity_id_y, dtype: int64

In [22]:
df_submission = pd.read_csv('/kaggle/input/multi-model-dataset/For validation/superAI/sample_submission.csv')
df_submission.loc[3:,'identity_id']=df_sub['identity_id_y'][3:]
df_submission

,caption_id,identity_id
0,35,41.0
1,69,38.0
2,42,44.0
3,17,46.0
4,76,39.0
...,...,...
95,29,43.0
96,8,44.0
97,9,41.0
98,64,48.0


In [23]:
df_submission.to_csv('submission.csv',index=False)

In [24]:
df_submission['identity_id'].value_counts()

46.0    11
41.0     8
38.0     8
44.0     8
39.0     8
40.0     8
45.0     8
47.0     8
43.0     8
42.0     7
48.0     6
49.0     5
51.0     3
55.0     2
62.0     1
50.0     1
Name: identity_id, dtype: int64